## Horse Survival Prediction

Given *medical data about horses*, let's try to predict whether a given horse will **survive** or not.

We will use a decision tree classifier and a random forest classifier to make our predictions.

Data source: https://www.kaggle.com/datasets/uciml/horse-colic

### Getting Started 

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [2]:
data = pd.read_csv('archive/horse.csv')
data

,surgery,age,hospital_number,rectal_temp,pulse,respiratory_rate,temp_of_extremities,peripheral_pulse,mucous_membrane,capillary_refill_time,pain,peristalsis,abdominal_distention,nasogastric_tube,nasogastric_reflux,nasogastric_reflux_ph,rectal_exam_feces,abdomen,packed_cell_volume,total_protein,abdomo_appearance,abdomo_protein,outcome,surgical_lesion,lesion_1,lesion_2,lesion_3,cp_data
0,no,adult,530101,38.5,66.0,28.0,cool,reduced,NaN,more_3_sec,extreme_pain,absent,severe,NaN,NaN,NaN,decreased,distend_large,45.0,8.4,NaN,NaN,died,no,11300,0,0,no
1,yes,adult,534817,39.2,88.0,20.0,NaN,NaN,pale_cyanotic,less_3_sec,mild_pain,absent,slight,NaN,NaN,NaN,absent,other,50.0,85.0,cloudy,2.0,euthanized,no,2208,0,0,no
2,no,adult,530334,38.3,40.0,24.0,normal,normal,pale_pink,less_3_sec,mild_pain,hypomotile,none,NaN,NaN,NaN,normal,normal,33.0,6.7,NaN,NaN,lived,no,0,0,0,yes
3,yes,young,5290409,39.1,164.0,84.0,cold,normal,dark_cyanotic,more_3_sec,depressed,absent,severe,none,less_1_liter,5.0,decreased,NaN,48.0,7.2,serosanguious,5.3,died,yes,2208,0,0,yes
4,no,adult,530255,37.3,104.0,35.0,NaN,NaN,dark_cyanotic,more_3_sec,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,74.0,7.4,NaN,NaN,died,no,4300,0,0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,yes,adult,533886,NaN,120.0,70.0,cold,NaN,pale_cyanotic,more_3_sec,depressed,absent,NaN,NaN,NaN,NaN,NaN,distend_large,55.0,65.0,NaN,NaN,euthanized,no,3205,0,0,no
295,no,adult,527702,37.2,72.0,24.0,cool,increased,pale_cyanotic,more_3_sec,severe_pain,hypomotile,moderate,significant,none,NaN,absent,distend_small,44.0,NaN,serosanguious,3.3,euthanized,yes,2208,0,0,yes
296,yes,adult,529386,37.5,72.0,30.0,cold,reduced,pale_cyanotic,less_3_sec,severe_pain,absent,moderate,slight,none,NaN,decreased,distend_large,60.0,6.8,NaN,NaN,died,yes,3205,0,0,no
297,yes,adult,530612,36.5,100.0,24.0,cool,reduced,pale_pink,less_3_sec,mild_pain,hypomotile,moderate,significant,none,NaN,absent,distend_small,50.0,6.0,serosanguious,3.4,lived,yes,2208,0,0,yes


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 28 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   surgery                299 non-null    object 
 1   age                    299 non-null    object 
 2   hospital_number        299 non-null    int64  
 3   rectal_temp            239 non-null    float64
 4   pulse                  275 non-null    float64
 5   respiratory_rate       241 non-null    float64
 6   temp_of_extremities    243 non-null    object 
 7   peripheral_pulse       230 non-null    object 
 8   mucous_membrane        252 non-null    object 
 9   capillary_refill_time  267 non-null    object 
 10  pain                   244 non-null    object 
 11  peristalsis            255 non-null    object 
 12  abdominal_distention   243 non-null    object 
 13  nasogastric_tube       195 non-null    object 
 14  nasogastric_reflux     193 non-null    object 
 15  nasoga

### Preprocessing

In [4]:
df = data.copy()

In [5]:
{column: list(df[column].unique()) for column in df.select_dtypes('object').columns}

{'surgery': ['no', 'yes'],
 'age': ['adult', 'young'],
 'temp_of_extremities': ['cool', nan, 'normal', 'cold', 'warm'],
 'peripheral_pulse': ['reduced', nan, 'normal', 'absent', 'increased'],
 'mucous_membrane': [nan,
  'pale_cyanotic',
  'pale_pink',
  'dark_cyanotic',
  'normal_pink',
  'bright_red',
  'bright_pink'],
 'capillary_refill_time': ['more_3_sec', 'less_3_sec', nan, '3'],
 'pain': ['extreme_pain',
  'mild_pain',
  'depressed',
  nan,
  'severe_pain',
  'alert'],
 'peristalsis': ['absent', 'hypomotile', nan, 'hypermotile', 'normal'],
 'abdominal_distention': ['severe', 'slight', 'none', nan, 'moderate'],
 'nasogastric_tube': [nan, 'none', 'slight', 'significant'],
 'nasogastric_reflux': [nan, 'less_1_liter', 'none', 'more_1_liter'],
 'rectal_exam_feces': ['decreased', 'absent', 'normal', nan, 'increased'],
 'abdomen': ['distend_large', 'other', 'normal', nan, 'firm', 'distend_small'],
 'abdomo_appearance': [nan, 'cloudy', 'serosanguious', 'clear'],
 'outcome': ['died', 'eut

In [6]:
# Categorize categorical features
binary_features = [
    'surgery',
    'age',
    'surgical_lesion',
    'cp_data'
]

positive_values = [
    'yes',
    'adult',
    'yes',
    'yes'
]

ordinal_features = [
    'temp_of_extremities',
    'peripheral_pulse',
    'capillary_refill_time',
    'pain',
    'peristalsis',
    'abdominal_distention',
    'nasogastric_tube',
    'nasogastric_reflux',
    'rectal_exam_feces'
]

orderings = [
    ['cold', 'cool', 'normal', 'warm'],
    ['absent', 'reduced', 'normal', 'increased'],
    ['less_3_sec', '3', 'more_3_sec'],
    ['alert', 'depressed', 'mild_pain', 'severe_pain', 'extreme_pain'],
    ['absent', 'hypomotile', 'normal', 'hypermotile'],
    ['none', 'slight', 'moderate', 'severe'],
    ['none', 'slight', 'significant'],
    ['none', 'less_1_liter', 'more_1_liter'],
    ['absent', 'decreased', 'normal', 'increased']
]

nominal_features = [
    'mucous_membrane',
    'abdomen',
    'abdomo_appearance',
    'hospital_number'
]

prefixes = [
    'MM',
    'AB', 
    'AA',
    'HN'
]

In [7]:
def binary_encode(df, columns, positive_values):
    df = df.copy()
    for column, positive_value in zip(columns, positive_values):
        df[column] = df[column].apply(lambda x: 1 if x == positive_value else 0)
    return df

def ordinal_encode(df, columns, orderings):
    df = df.copy()
    for column, ordering in zip(columns, orderings):
        df[column] = df[column].apply(lambda x: ordering.index(x))
    return df
        
def onehot_encode(df, columns, prefixes):
    df = df.copy()
    for column, prefix in zip(columns, prefixes):
        dummies = pd.get_dummies(df[column], prefix=prefix, dtype=int)
        df = pd.concat([df, dummies], axis=1)
        df = df.drop(column, axis=1)
    return df

In [8]:
# Missing values
df.isna().sum()

surgery                    0
age                        0
hospital_number            0
rectal_temp               60
pulse                     24
respiratory_rate          58
temp_of_extremities       56
peripheral_pulse          69
mucous_membrane           47
capillary_refill_time     32
pain                      55
peristalsis               44
abdominal_distention      56
nasogastric_tube         104
nasogastric_reflux       106
nasogastric_reflux_ph    246
rectal_exam_feces        102
abdomen                  118
packed_cell_volume        29
total_protein             33
abdomo_appearance        165
abdomo_protein           198
outcome                    0
surgical_lesion            0
lesion_1                   0
lesion_2                   0
lesion_3                   0
cp_data                    0
dtype: int64

In [9]:
# Fill missing values 
for column in df.columns:
    if column in df.select_dtypes('object').columns:
        if column not in nominal_features:
            df[column] = df[column].fillna(df[column].mode()[0])
    else:
        df[column] = df[column].fillna(df[column].mean())

In [10]:
df.isna().sum()

surgery                    0
age                        0
hospital_number            0
rectal_temp                0
pulse                      0
respiratory_rate           0
temp_of_extremities        0
peripheral_pulse           0
mucous_membrane           47
capillary_refill_time      0
pain                       0
peristalsis                0
abdominal_distention       0
nasogastric_tube           0
nasogastric_reflux         0
nasogastric_reflux_ph      0
rectal_exam_feces          0
abdomen                  118
packed_cell_volume         0
total_protein              0
abdomo_appearance        165
abdomo_protein             0
outcome                    0
surgical_lesion            0
lesion_1                   0
lesion_2                   0
lesion_3                   0
cp_data                    0
dtype: int64

In [11]:
# Encode categorical feature columns
df = binary_encode(df, binary_features, positive_values=positive_values)
df = ordinal_encode(df, ordinal_features, orderings=orderings)
df = onehot_encode(df, nominal_features, prefixes=prefixes)

In [12]:
df

,surgery,age,rectal_temp,pulse,respiratory_rate,temp_of_extremities,peripheral_pulse,capillary_refill_time,pain,peristalsis,abdominal_distention,nasogastric_tube,nasogastric_reflux,nasogastric_reflux_ph,rectal_exam_feces,packed_cell_volume,total_protein,abdomo_protein,outcome,surgical_lesion,lesion_1,lesion_2,lesion_3,cp_data,MM_bright_pink,MM_bright_red,MM_dark_cyanotic,MM_normal_pink,MM_pale_cyanotic,MM_pale_pink,AB_distend_large,AB_distend_small,AB_firm,AB_normal,AB_other,AA_clear,AA_cloudy,AA_serosanguious,HN_518476,HN_521399,HN_521681,HN_522979,HN_523190,HN_526639,HN_526802,HN_527365,HN_527463,HN_527518,HN_527524,HN_527544,HN_527563,HN_527677,HN_527698,HN_527702,HN_527706,HN_527709,HN_527734,HN_527758,HN_527829,HN_527883,HN_527916,HN_527927,HN_527929,HN_527933,HN_527940,HN_527957,HN_528006,HN_528019,HN_528031,HN_528047,HN_528134,HN_528151,HN_528169,HN_528178,HN_528179,HN_528183,HN_528214,HN_528247,HN_528248,HN_528298,HN_528299,HN_528305,HN_528355,HN_528461,HN_528469,HN_528503,HN_528523,HN_528548,HN_528570,HN_528590,HN_528620,HN_528630,HN_528638,HN_528641,HN_528653,HN_528668,HN_528682,HN_528702,HN_528729,HN_528742,HN_528743,HN_528800,HN_528804,HN_528812,HN_528872,HN_528890,HN_528904,HN_528931,HN_528964,HN_528977,HN_528996,HN_529045,HN_529126,HN_529135,HN_529160,HN_529172,HN_529183,HN_529272,HN_529296,HN_529304,HN_529340,HN_529373,HN_529386,HN_529388,HN_529399,HN_529424,HN_529427,HN_529428,HN_529461,HN_529475,HN_529483,HN_529493,HN_529498,HN_529518,HN_529528,HN_529607,HN_529628,HN_529640,HN_529642,HN_529663,HN_529667,HN_529685,HN_529703,HN_529729,HN_529736,HN_529764,HN_529766,HN_529777,HN_529796,HN_529812,HN_529821,HN_529827,HN_529840,HN_529849,HN_529865,HN_529888,HN_529893,HN_529960,HN_530001,HN_530002,HN_530028,HN_530034,HN_530051,HN_530101,HN_530157,HN_530170,HN_530233,HN_530239,HN_530242,HN_530251,HN_530254,HN_530255,HN_530276,HN_530294,HN_530297,HN_530301,HN_530310,HN_530319,HN_530334,HN_530354,HN_530360,HN_530366,HN_530381,HN_530384,HN_530401,HN_530402,HN_530431,HN_530438,HN_530439,HN_530478,HN_530526,HN_530544,HN_530561,HN_530612,HN_530624,HN_530670,HN_530693,HN_532110,HN_532349,HN_532985,HN_533692,HN_533696,HN_533697,HN_533723,HN_533736,HN_533738,HN_533750,HN_533793,HN_533836,HN_533847,HN_533871,HN_533885,HN_533886,HN_533887,HN_533902,HN_533928,HN_533942,HN_533968,HN_533983,HN_534004,HN_534053,HN_534069,HN_534073,HN_534092,HN_534115,HN_534135,HN_534145,HN_534157,HN_534163,HN_534183,HN_534197,HN_534280,HN_534293,HN_534324,HN_534403,HN_534478,HN_534491,HN_534519,HN_534556,HN_534579,HN_534597,HN_534615,HN_534618,HN_534624,HN_534626,HN_534644,HN_534719,HN_534753,HN_534756,HN_534784,HN_534787,HN_534788,HN_534817,HN_534833,HN_534857,HN_534885,HN_534899,HN_534917,HN_534925,HN_534933,HN_534938,HN_534963,HN_534998,HN_535029,HN_535031,HN_535043,HN_535054,HN_535085,HN_535130,HN_535137,HN_535158,HN_535163,HN_535166,HN_535176,HN_535196,HN_535208,HN_535240,HN_535246,HN_535292,HN_535314,HN_535330,HN_535338,HN_535364,HN_535381,HN_535392,HN_535407,HN_535415,HN_5262541,HN_5262543,HN_5275212,HN_5277409,HN_5278331,HN_5279441,HN_5279442,HN_5279822,HN_5281091,HN_5282839,HN_5283431,HN_5287179,HN_5287279,HN_5288249,HN_5289419,HN_5290409,HN_5290481,HN_5290482,HN_5290759,HN_5291329,HN_5291409,HN_5291719,HN_5292489,HN_5292929,HN_5294369,HN_5294539,HN_5297159,HN_5297379,HN_5299253,HN_5299603,HN_5299629,HN_5301219,HN_5305129,HN_5305629
0,0,1,38.500000,66.0,28.0,1,1,2,4,0,3,1,0,4.707547,1,45.0,8.400000,3.039604,died,0,11300,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [13]:
# Split df into X and y
y = df['outcome'].copy()
X = df.drop('outcome', axis=1).copy()

In [14]:
# Encode labels
label_mapping = {
    'lived': 0,
    'died': 1,
    'euthanized': 2
}

y = y.replace(label_mapping)

C:\Users\dukea_6wbfoy1\AppData\Local\Temp\ipykernel_11000\3824003214.py:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y = y.replace(label_mapping)


In [15]:
y

0      1
1      2
2      0
3      1
4      1
      ..
294    2
295    2
296    1
297    0
298    2
Name: outcome, Length: 299, dtype: int64

In [16]:
X

,surgery,age,rectal_temp,pulse,respiratory_rate,temp_of_extremities,peripheral_pulse,capillary_refill_time,pain,peristalsis,abdominal_distention,nasogastric_tube,nasogastric_reflux,nasogastric_reflux_ph,rectal_exam_feces,packed_cell_volume,total_protein,abdomo_protein,surgical_lesion,lesion_1,lesion_2,lesion_3,cp_data,MM_bright_pink,MM_bright_red,MM_dark_cyanotic,MM_normal_pink,MM_pale_cyanotic,MM_pale_pink,AB_distend_large,AB_distend_small,AB_firm,AB_normal,AB_other,AA_clear,AA_cloudy,AA_serosanguious,HN_518476,HN_521399,HN_521681,HN_522979,HN_523190,HN_526639,HN_526802,HN_527365,HN_527463,HN_527518,HN_527524,HN_527544,HN_527563,HN_527677,HN_527698,HN_527702,HN_527706,HN_527709,HN_527734,HN_527758,HN_527829,HN_527883,HN_527916,HN_527927,HN_527929,HN_527933,HN_527940,HN_527957,HN_528006,HN_528019,HN_528031,HN_528047,HN_528134,HN_528151,HN_528169,HN_528178,HN_528179,HN_528183,HN_528214,HN_528247,HN_528248,HN_528298,HN_528299,HN_528305,HN_528355,HN_528461,HN_528469,HN_528503,HN_528523,HN_528548,HN_528570,HN_528590,HN_528620,HN_528630,HN_528638,HN_528641,HN_528653,HN_528668,HN_528682,HN_528702,HN_528729,HN_528742,HN_528743,HN_528800,HN_528804,HN_528812,HN_528872,HN_528890,HN_528904,HN_528931,HN_528964,HN_528977,HN_528996,HN_529045,HN_529126,HN_529135,HN_529160,HN_529172,HN_529183,HN_529272,HN_529296,HN_529304,HN_529340,HN_529373,HN_529386,HN_529388,HN_529399,HN_529424,HN_529427,HN_529428,HN_529461,HN_529475,HN_529483,HN_529493,HN_529498,HN_529518,HN_529528,HN_529607,HN_529628,HN_529640,HN_529642,HN_529663,HN_529667,HN_529685,HN_529703,HN_529729,HN_529736,HN_529764,HN_529766,HN_529777,HN_529796,HN_529812,HN_529821,HN_529827,HN_529840,HN_529849,HN_529865,HN_529888,HN_529893,HN_529960,HN_530001,HN_530002,HN_530028,HN_530034,HN_530051,HN_530101,HN_530157,HN_530170,HN_530233,HN_530239,HN_530242,HN_530251,HN_530254,HN_530255,HN_530276,HN_530294,HN_530297,HN_530301,HN_530310,HN_530319,HN_530334,HN_530354,HN_530360,HN_530366,HN_530381,HN_530384,HN_530401,HN_530402,HN_530431,HN_530438,HN_530439,HN_530478,HN_530526,HN_530544,HN_530561,HN_530612,HN_530624,HN_530670,HN_530693,HN_532110,HN_532349,HN_532985,HN_533692,HN_533696,HN_533697,HN_533723,HN_533736,HN_533738,HN_533750,HN_533793,HN_533836,HN_533847,HN_533871,HN_533885,HN_533886,HN_533887,HN_533902,HN_533928,HN_533942,HN_533968,HN_533983,HN_534004,HN_534053,HN_534069,HN_534073,HN_534092,HN_534115,HN_534135,HN_534145,HN_534157,HN_534163,HN_534183,HN_534197,HN_534280,HN_534293,HN_534324,HN_534403,HN_534478,HN_534491,HN_534519,HN_534556,HN_534579,HN_534597,HN_534615,HN_534618,HN_534624,HN_534626,HN_534644,HN_534719,HN_534753,HN_534756,HN_534784,HN_534787,HN_534788,HN_534817,HN_534833,HN_534857,HN_534885,HN_534899,HN_534917,HN_534925,HN_534933,HN_534938,HN_534963,HN_534998,HN_535029,HN_535031,HN_535043,HN_535054,HN_535085,HN_535130,HN_535137,HN_535158,HN_535163,HN_535166,HN_535176,HN_535196,HN_535208,HN_535240,HN_535246,HN_535292,HN_535314,HN_535330,HN_535338,HN_535364,HN_535381,HN_535392,HN_535407,HN_535415,HN_5262541,HN_5262543,HN_5275212,HN_5277409,HN_5278331,HN_5279441,HN_5279442,HN_5279822,HN_5281091,HN_5282839,HN_5283431,HN_5287179,HN_5287279,HN_5288249,HN_5289419,HN_5290409,HN_5290481,HN_5290482,HN_5290759,HN_5291329,HN_5291409,HN_5291719,HN_5292489,HN_5292929,HN_5294369,HN_5294539,HN_5297159,HN_5297379,HN_5299253,HN_5299603,HN_5299629,HN_5301219,HN_5305129,HN_5305629
0,0,1,38.500000,66.0,28.0,1,1,2,4,0,3,1,0,4.707547,1,45.0,8.400000,3.039604,0,11300,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [18]:
# Scale X with a standard scaler
scaler = StandardScaler()

X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

In [19]:
X

,surgery,age,rectal_temp,pulse,respiratory_rate,temp_of_extremities,peripheral_pulse,capillary_refill_time,pain,peristalsis,abdominal_distention,nasogastric_tube,nasogastric_reflux,nasogastric_reflux_ph,rectal_exam_feces,packed_cell_volume,total_protein,abdomo_protein,surgical_lesion,lesion_1,lesion_2,lesion_3,cp_data,MM_bright_pink,MM_bright_red,MM_dark_cyanotic,MM_normal_pink,MM_pale_cyanotic,MM_pale_pink,AB_distend_large,AB_distend_small,AB_firm,AB_normal,AB_other,AA_clear,AA_cloudy,AA_serosanguious,HN_518476,HN_521399,HN_521681,HN_522979,HN_523190,HN_526639,HN_526802,HN_527365,HN_527463,HN_527518,HN_527524,HN_527544,HN_527563,HN_527677,HN_527698,HN_527702,HN_527706,HN_527709,HN_527734,HN_527758,HN_527829,HN_527883,HN_527916,HN_527927,HN_527929,HN_527933,HN_527940,HN_527957,HN_528006,HN_528019,HN_528031,HN_528047,HN_528134,HN_528151,HN_528169,HN_528178,HN_528179,HN_528183,HN_528214,HN_528247,HN_528248,HN_528298,HN_528299,HN_528305,HN_528355,HN_528461,HN_528469,HN_528503,HN_528523,HN_528548,HN_528570,HN_528590,HN_528620,HN_528630,HN_528638,HN_528641,HN_528653,HN_528668,HN_528682,HN_528702,HN_528729,HN_528742,HN_528743,HN_528800,HN_528804,HN_528812,HN_528872,HN_528890,HN_528904,HN_528931,HN_528964,HN_528977,HN_528996,HN_529045,HN_529126,HN_529135,HN_529160,HN_529172,HN_529183,HN_529272,HN_529296,HN_529304,HN_529340,HN_529373,HN_529386,HN_529388,HN_529399,HN_529424,HN_529427,HN_529428,HN_529461,HN_529475,HN_529483,HN_529493,HN_529498,HN_529518,HN_529528,HN_529607,HN_529628,HN_529640,HN_529642,HN_529663,HN_529667,HN_529685,HN_529703,HN_529729,HN_529736,HN_529764,HN_529766,HN_529777,HN_529796,HN_529812,HN_529821,HN_529827,HN_529840,HN_529849,HN_529865,HN_529888,HN_529893,HN_529960,HN_530001,HN_530002,HN_530028,HN_530034,HN_530051,HN_530101,HN_530157,HN_530170,HN_530233,HN_530239,HN_530242,HN_530251,HN_530254,HN_530255,HN_530276,HN_530294,HN_530297,HN_530301,HN_530310,HN_530319,HN_530334,HN_530354,HN_530360,HN_530366,HN_530381,HN_530384,HN_530401,HN_530402,HN_530431,HN_530438,HN_530439,HN_530478,HN_530526,HN_530544,HN_530561,HN_530612,HN_530624,HN_530670,HN_530693,HN_532110,HN_532349,HN_532985,HN_533692,HN_533696,HN_533697,HN_533723,HN_533736,HN_533738,HN_533750,HN_533793,HN_533836,HN_533847,HN_533871,HN_533885,HN_533886,HN_533887,HN_533902,HN_533928,HN_533942,HN_533968,HN_533983,HN_534004,HN_534053,HN_534069,HN_534073,HN_534092,HN_534115,HN_534135,HN_534145,HN_534157,HN_534163,HN_534183,HN_534197,HN_534280,HN_534293,HN_534324,HN_534403,HN_534478,HN_534491,HN_534519,HN_534556,HN_534579,HN_534597,HN_534615,HN_534618,HN_534624,HN_534626,HN_534644,HN_534719,HN_534753,HN_534756,HN_534784,HN_534787,HN_534788,HN_534817,HN_534833,HN_534857,HN_534885,HN_534899,HN_534917,HN_534925,HN_534933,HN_534938,HN_534963,HN_534998,HN_535029,HN_535031,HN_535043,HN_535054,HN_535085,HN_535130,HN_535137,HN_535158,HN_535163,HN_535166,HN_535176,HN_535196,HN_535208,HN_535240,HN_535246,HN_535292,HN_535314,HN_535330,HN_535338,HN_535364,HN_535381,HN_535392,HN_535407,HN_535415,HN_5262541,HN_5262543,HN_5275212,HN_5277409,HN_5278331,HN_5279441,HN_5279442,HN_5279822,HN_5281091,HN_5282839,HN_5283431,HN_5287179,HN_5287279,HN_5288249,HN_5289419,HN_5290409,HN_5290481,HN_5290482,HN_5290759,HN_5291329,HN_5291409,HN_5291719,HN_5292489,HN_5292929,HN_5294369,HN_5294539,HN_5297159,HN_5297379,HN_5299253,HN_5299603,HN_5299629,HN_5301219,HN_5305129,HN_5305629
0,-1.229880,0.295420,0.506209,-0.218798,-0.155463,-0.473504,-1.088914,1.675999,1.731538,-1.185885,1.823732,0.298823,-0.536637,0.000000,0.352925,-0.132070,-6.162094e-01,0.000000,-1.320272,1.415021,-0.139371,-0.057928,-0.703562,-0.333952,-0.302061,-0.26774,-0.599242,-0.398641,-0.485322,1.683251,-0.409840,-0.213201,-0.321436,-0.260494,-0.398641,-0.431866,-0.426401,-0.057928,-0.057928,-0.057928,-0.057928,-0.057928,-0.057928,-0.057928,-0.057928,-0.057928,-0.057928,-0.057928,-0.082061,-0.057928,-0.057928,-0.057928,-0.057928,-0.057928,-0.057928,-0.057928,-0.057928,-0.057928,-0.057928,-0.082061,-0.057928,-0.057928,-0.057928,-0.0

### Training

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=123)

In [21]:
model = DecisionTreeClassifier()

model.fit(X_train, y_train)

,criterion,'gini'
,splitter,'best'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,None
,random_state,None
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,class_weight,None


In [22]:
print("Decision Tree Accuracy: {:.2f}%".format(model.score(X_test, y_test) * 100))

Decision Tree Accuracy: 61.11%


In [23]:
ensemble_model = RandomForestClassifier()

ensemble_model.fit(X_train, y_train)

,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [24]:
print("Random Forest Accuracy: {:.2f}%".format(ensemble_model.score(X_test, y_test) * 100))

Random Forest Accuracy: 72.22%
